Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [901]:
from random import choices, choice, random, randint, uniform
from copy import copy, deepcopy

import lab9_lib
from dataclasses import dataclass

In [902]:
fitness = lab9_lib.make_problem(10)
'''
print("Massimo, ", fitness.x)
for n in range(10):
    ind = choices([0, 1], k=50)
    print(f"{''.join(str(g) for g in ind)}: {fitness(ind):.2%}")

print(fitness.calls)
'''

'\nprint("Massimo, ", fitness.x)\nfor n in range(10):\n    ind = choices([0, 1], k=50)\n    print(f"{\'\'.join(str(g) for g in ind)}: {fitness(ind):.2%}")\n\nprint(fitness.calls)\n'

# GA aproach

In [903]:
# variables
GENOTYPE_SIZE = 50
POPULATION_SIZE = 90
OFFSPRING_SIZE = 30
TOURNAMENT_SIZE = 2
MUTATION_PROBABILITY = .15

In [904]:
#Individual class and methods
@dataclass
class Individual:
    fitness: None
    genotype: list[int]

def elitism_selection(population, num_elites=1):
    # Sort by fitness in descending order
    sorted_population = sorted(population, key=lambda x: x.fitness, reverse=True)
    
    # Select the top individuals (elites)
    elites = sorted_population[:num_elites]
    
    return elites

def roulette_wheel_selection(population):
    # Calculate total fitness
    total_fitness = 0
    for ind in population:
        total_fitness += ind.fitness
    
    # Generate a random number between 0 and the total fitness
    spin = uniform(0, total_fitness)

    # Perform the selection
    current_sum = 0
    for ind in population:
        current_sum += ind.fitness
        if current_sum >= spin:
            return ind

def tournament_selection(pop):
    pool = [choice(pop) for _ in range(TOURNAMENT_SIZE)]
    champion = max(pool, key=lambda i: i.fitness)
    return champion

def mutate(ind: Individual) -> Individual:
    offspring = copy(ind)
    pos = randint(0, GENOTYPE_SIZE-1)
    offspring.genotype[pos] = 1 - offspring.genotype[pos]
    offspring.fitness = None
    return offspring

def one_cut_xover(ind1: Individual, ind2: Individual) -> (Individual, Individual):
    cut_point = randint(0, GENOTYPE_SIZE-1)
    offspring_1 = Individual(fitness=None,
                           genotype=ind1.genotype[:cut_point] + ind2.genotype[cut_point:])
    offspring_2 = Individual(fitness=None,
                        genotype=ind2.genotype[:cut_point] + ind1.genotype[cut_point:])
    
    assert (len(offspring_1.genotype) == GENOTYPE_SIZE) and (len(offspring_2.genotype) == GENOTYPE_SIZE)
    return offspring_1, offspring_2

def two_cut_xover(ind1: Individual, ind2: Individual) -> (Individual, Individual):
    #check that I have two genotypes made up of at least 3 elements
    assert len(ind1.genotype) > 2 and len(ind2.genotype) > 2
    
    #fisrt cut can't be the first element and neither the 2 before the end (end included)
    first_cut = randint(1, len(ind2.genotype)-2)
    #second cut can't be the second and neither the 1 before the end (end included)
    second_cut = randint(first_cut+1, len(ind2.genotype)-1)
    #print(first_cut, second_cut)

    #create the new child with the new cut
    offspring_1 = Individual(fitness=None,
                           genotype=ind1.genotype[:first_cut] + ind2.genotype[first_cut:second_cut] + ind1.genotype[second_cut:])
    #create the new child with the new cut
    offspring_2 = Individual(fitness=None,
                           genotype=ind2.genotype[:first_cut] + ind1.genotype[first_cut:second_cut] + ind2.genotype[second_cut:])

    assert (len(offspring_1.genotype) == GENOTYPE_SIZE) and (len(offspring_2.genotype) == GENOTYPE_SIZE)
    return offspring_1, offspring_2

def three_cut_xover(ind1: Individual, ind2: Individual) -> (Individual, Individual):
    #check that I have two genotypes made up of at least 4 elements
    assert len(ind1.genotype) > 3 and len(ind2.genotype) > 3
    
    #fisrt cut can't be the first element and neither the 3 before the end (end included)
    first_cut = randint(1, len(ind2.genotype)-3)
    #second cut can't be the second and neither the 2 before the end (end included)
    second_cut = randint(first_cut+1, len(ind2.genotype)-2)
    #third cut can't be the second and neither the 1 before the end (end included)
    third_cut = randint(second_cut+1, len(ind2.genotype)-1)
    #print(first_cut, second_cut, third_cut)

    #create the new child with the new cut
    offspring_1 = Individual(fitness=None,
                           genotype=ind1.genotype[:first_cut] + ind2.genotype[first_cut:second_cut] +
                                    ind1.genotype[second_cut:third_cut] + ind2.genotype[third_cut:])
    #create the new child with the new cut
    offspring_2 = Individual(fitness=None,
                           genotype=ind2.genotype[:first_cut] + ind1.genotype[first_cut:second_cut] +
                                    ind2.genotype[second_cut:third_cut] + ind1.genotype[third_cut:])
    
    assert (len(offspring_1.genotype) == GENOTYPE_SIZE) and (len(offspring_2.genotype) == GENOTYPE_SIZE)
    return offspring_1, offspring_2

def uniform_xover(ind1: Individual, ind2: Individual) -> (Individual, Individual):
    #check that I have two genotypes of the same lenght
    assert len(ind1.genotype) == len(ind2.genotype) 
    
    #create the new child with the new cut
    offspring_1 = Individual(fitness=None, genotype=[])
    #create the new child with the new cut
    offspring_2 = Individual(fitness=None, genotype=[])

    #fill the two of their genotype
    for i in range(GENOTYPE_SIZE):
        if(choice((0, 1)) == 1):
            offspring_1.genotype.append(ind1.genotype[i])
            offspring_2.genotype.append(ind2.genotype[i])
        else:
            offspring_1.genotype.append(ind2.genotype[i])
            offspring_2.genotype.append(ind1.genotype[i])

    assert (len(offspring_1.genotype) == GENOTYPE_SIZE) and (len(offspring_2.genotype) == GENOTYPE_SIZE)
    return offspring_1, offspring_2

#put all xover in a list
xover_functions = [one_cut_xover, two_cut_xover, three_cut_xover, uniform_xover]

In [905]:
#start the population 
population = [
    Individual(
        genotype = choices([0, 1], k=50),
        fitness = None,
    )
    for _ in range(POPULATION_SIZE)
]
#add the fitness to every individuals
for i in population:
    i.fitness = fitness(i.genotype)

In [906]:
#First algorithm

# Variable for exit if the fitness is stationary
# The first element of the list represents the previous fitness, and the second element is the number of consecutive times it has been observed.
last_fitness = [0, 0]

while(False): #for generation in range(100):
    offspring = list()
    
    for counter in range(OFFSPRING_SIZE):
        if random() < MUTATION_PROBABILITY:  # self-adapt mutation probability
            # mutation  # add more clever mutations
            parent = roulette_wheel_selection(population)
            child = mutate(parent)
            #add new child to offspring
            offspring.append(child)
        else:
            # xover # add more xovers
            parent_1 = roulette_wheel_selection(population)
            parent_2 = roulette_wheel_selection(population)
            #choose random the crossover function to incress the diversity
            child_1, child_2 = xover_functions[randint(0, len(xover_functions)-1)](parent_1, parent_2)
            #add new children to offspring
            offspring.extend([child_1, child_2])

    for i in offspring:
        i.fitness = fitness(i.genotype)
        
    population.extend(offspring)
    population.sort(key=lambda i: i.fitness, reverse=True)
    population = population[:POPULATION_SIZE]

    # Check to halt the code in the event of a stationary minimum.
    if(population[0].fitness == last_fitness[0]):
        last_fitness[1]+=1
    else:
        last_fitness[1] = 0
    last_fitness[0] = population[0].fitness
    if(last_fitness[1] > 30):
        break
    
    print("calls:",fitness._calls, "\tfitness: ",population[0].fitness)


print("===>", fitness.calls)

===> 90


# Island Model

# Trash only for my own test

In [907]:
'''
calls = 0
genome = choices([0, 1], k=GENOTYPE_SIZE)
print("start genome", genome)
x = 10

def onemax(genome):
    return sum(bool(g) for g in genome)

for i in range(2):
    calls += 1
    fitnesses = sorted((onemax(genome[s :: x]) for s in range(x)), reverse=True)
    val = sum(f for f in fitnesses if f == fitnesses[0]) - sum(
        f * (0.1 ** (k + 1)) for k, f in enumerate(f for f in fitnesses if f < fitnesses[0])
    )
    print(" valore res", (val / len(genome)) )


#creazione inziale
prova = choices([0, 1], k=50)#[ el for el in range(49)]
#controllo fitness
fitness = sorted((onemax(genome[s :: x]) for s in range(x)), reverse=True)
print("Best fintess", fitness[0])
#creazione vettore con elementi minori del elemento maggiore
new_tmp=[]
for f in fitness:
    if f < fitness[0]:
        new_tmp.append(f)
        print("==> ", f)

#creazione di un valore che muove la virgola di 1 per ogni valore risultante nel vettore precedente
new_2 = []
for k, f in enumerate(new_tmp):
    abc = f * (0.1 ** (k + 1))
    new_2.append(abc)
    print("222=> ",abc)

#calcolo della somma dei valori precedentemente raccolti
somma = sum(new_2)
print(somma)

new_tmp_2 = []
for f in fitnesses:
    if f == fitnesses[0]:
        new_tmp_2.append(f)

somma_2 = sum(new_tmp_2)
print("seconda somma ", somma_2)

totale = somma_2 - somma
print("Differenza somme:", totale) 
'''

'\ncalls = 0\ngenome = choices([0, 1], k=GENOTYPE_SIZE)\nprint("start genome", genome)\nx = 10\n\ndef onemax(genome):\n    return sum(bool(g) for g in genome)\n\nfor i in range(2):\n    calls += 1\n    fitnesses = sorted((onemax(genome[s :: x]) for s in range(x)), reverse=True)\n    val = sum(f for f in fitnesses if f == fitnesses[0]) - sum(\n        f * (0.1 ** (k + 1)) for k, f in enumerate(f for f in fitnesses if f < fitnesses[0])\n    )\n    print(" valore res", (val / len(genome)) )\n\n\n#creazione inziale\nprova = choices([0, 1], k=50)#[ el for el in range(49)]\n#controllo fitness\nfitness = sorted((onemax(genome[s :: x]) for s in range(x)), reverse=True)\nprint("Best fintess", fitness[0])\n#creazione vettore con elementi minori del elemento maggiore\nnew_tmp=[]\nfor f in fitness:\n    if f < fitness[0]:\n        new_tmp.append(f)\n        print("==> ", f)\n\n#creazione di un valore che muove la virgola di 1 per ogni valore risultante nel vettore precedente\nnew_2 = []\nfor k, f 